In [1]:
import tensorflow as tf       # To work with USE4
import pandas as pd           # To work with tables 
import tensorflow_hub as hub  # contains USE4
import pickle
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #Model is imported from this URL
model = hub.load(module_url)

def embed(input):
  return model(input)

2023-10-15 18:09:33.462984: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-15 18:09:34.332484: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-15 18:09:35.266832: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-15 18:09:35.286462: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
Data = pd.read_csv("/home/madhav/textsimilarity/Semantic-Similarity-between-2-sentences/Precily_Text_Similarity.csv") # read the given data

In [3]:
Data.head()

,text1,text2
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...


In [4]:
Data.shape 

(3000, 2)

In [5]:
Data['text1'][0]

'broadband challenges tv viewing the number of europeans with broadband has exploded over the past 12 months  with the web eating into tv viewing habits  research suggests.  just over 54 million people are hooked up to the net via broadband  up from 34 million a year ago  according to market analysts nielsen/netratings. the total number of people online in europe has broken the 100 million mark. the popularity of the net has meant that many are turning away from tv  say analysts jupiter research. it found that a quarter of web users said they spent less time watching tv in favour of the net  the report by nielsen/netratings found that the number of people with fast internet access had risen by 60% over the past year.  the biggest jump was in italy  where it rose by 120%. britain was close behind  with broadband users almost doubling in a year. the growth has been fuelled by lower prices and a wider choice of always-on  fast-net subscription plans.  twelve months ago high speed internet

In [6]:
type(Data['text1'][0]) # check type of data

str

In [7]:
message = [Data['text1'][0], Data['text2'][0]] # creating a list combining 2 texts
message_embeddings = embed(message)
message_embeddings

2023-10-15 18:09:39.756092: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-10-15 18:09:40.812388: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


<tf.Tensor: shape=(2, 512), dtype=float32, numpy=
array([[-0.02720232,  0.00681648, -0.03939367, ..., -0.03903365,
        -0.05795865, -0.05810071],
       [-0.05569993, -0.05644849, -0.056383  , ...,  0.04282603,
        -0.05645383, -0.05647698]], dtype=float32)>

In [8]:
type(message_embeddings)

tensorflow.python.framework.ops.EagerTensor

In [9]:
type(message_embeddings[0])

tensorflow.python.framework.ops.EagerTensor

In [10]:
type(tf.make_ndarray(tf.make_tensor_proto(message_embeddings))) # as we need to convert the vector in the format of numpy array

numpy.ndarray

In [11]:
a_np = tf.make_ndarray(tf.make_tensor_proto(message_embeddings))

In [12]:
# Calculate cosine similarity using the below formula i.e ###cosin = dot(a,b)/norm(a)*norm(b)
# For each vector pair, we found the cosine between the by using usual cosine formula.
from numpy import dot                                           # to calculate the dot product of two vectors
from numpy.linalg import norm                                   #for finding the norm of a vector

ans = []                                                        # This list will contain the cosin similarity value for each vector pair present.
for i in range(len(Data)):
  messages = [Data['text1'][i], Data['text2'][i]]               #storing each sentence pair in messages
  message_embeddings = embed(messages)                          #converting the sentence pair to vector pair using the embed() function
  a = tf.make_ndarray(tf.make_tensor_proto(message_embeddings)) #storing the vector in the form of numpy array
  cos_sim = dot(a[0], a[1])/(norm(a[0])*norm(a[1]))             #Finding the cosine between the two vectors
  ans.append(cos_sim)                                           #Appending the values into the ans list

In [13]:
Ans = pd.DataFrame(ans, columns = ['Similarity_Score'])  

In [14]:
Ans.head()

,Similarity_Score
0,0.272669
1,0.277622
2,0.169011
3,0.157467
4,0.246201


In [15]:
Data = Data.join(Ans)

In [16]:
# since we get the value ranging from -1 to 1 from cosine similarity. But, we need values ranging from 0 to 1 hence we will add 1 to the cosine similarity value and then normalizze it. 
Data['Similarity_Score'] = Data['Similarity_Score'] + 1   

In [17]:
Data.head(2)

,text1,text2,Similarity_Score
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...,1.272669
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...,1.277622


In [18]:
Data['Similarity_Score'] = Data['Similarity_Score']/Data['Similarity_Score'].abs().max() #Normalizing the Similarity_Score to get the value between 0 and 1

In [19]:
Data.head()

,text1,text2,Similarity_Score
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...,0.636334
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...,0.638811
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...,0.584505
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...,0.578734
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...,0.623100


In [20]:
Submission = Data[['Similarity_Score']]

In [21]:
Submission.head()

,Similarity_Score
0,0.636334
1,0.638811
2,0.584505
3,0.578734
4,0.623100


In [22]:
Submission.to_csv('Submission.csv') 